In [1]:
%gui qt
import napari
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from shapely.geometry import Polygon, Point,LineString,  mapping


In [2]:
import json
import geojson
from spacetx_spatial_analysis import depth_coordinate

## Make a napari viewer...

In [3]:
v = napari.Viewer()

## Load spot table from 1 dataset

In [ ]:
# load one spotxgene table
all_spots = pd.read_csv("/Users/brian/Downloads/spaceTX_excitatory_Linnarsson_Lab.csv")

In [7]:
genes = all_spots["gene"].unique()

In [9]:
all_spots

,gene,round_num,dot_id,r_px,c_px,r_um,c_um
0,Atp2b4,7,0_0,21497.072445,8137.101776,2325.774584,880.355432
1,Atp2b4,7,0_2,21491.072445,8194.101776,2325.125442,886.522279
2,Atp2b4,7,0_4,21484.072445,8168.101776,2324.368110,883.709331
3,Atp2b4,7,0_5,21482.072445,8199.101776,2324.151729,887.063230
4,Atp2b4,7,0_6,21481.072445,8191.101776,2324.043539,886.197708
...,...,...,...,...,...,...,...
1543502,Zfp827,3,95_1063,60.201154,26161.498175,6.513181,2830.420174
1543503,Zfp827,3,95_1066,50.201154,26147.498175,5.431278,2828.905510
1543504,Zfp827,3,95_1068,36.201154,26849.498175,3.916614,2904.855096
1543505,Zfp827,3,95_1076,16.201154,25931.498175,1.752808,2805.536406


In [8]:
subsample = 1 # change to >1 if number of points is too high
colorlist = ["magenta", "red","blue","cyan", "green", "orange","red"]


In [10]:
cindex = 0
for ii, gene_to_plot in enumerate(["Gad1","Gad2","Slc17a7","Cux2","Calb1","Meis2","Foxp2", "Atp1a3" , "Vxn", "Pcp4" , "Fezf2" , "Rorb" , "Gm11549"]):

    if gene_to_plot in genes:
        if gene_to_plot == "Slc17a7":
            spots = v.add_points(all_spots[all_spots["gene"]==gene_to_plot][["r_um","c_um"]].values[::subsample,:], symbol='disc',
                                 name = gene_to_plot+"+",edge_color= [0,0,0,0], size = 1.5,
                                 face_color= colorlist[ cindex % len(colorlist)] , blending = "translucent", opacity = 0.8)
        else:
            spots = v.add_points(all_spots[all_spots["gene"]==gene_to_plot][["r_um","c_um"]].values[::subsample,:], symbol='disc',
                     name = gene_to_plot+"+",edge_color= [0,0,0,0], size = 2.,
                     face_color= colorlist[ cindex % len(colorlist)] , blending = "translucent", opacity = 0.7)
        cindex = cindex + 1
    if gene_to_plot.lower() in genes:
        
        if gene_to_plot == "Slc17a7":
            spots = v.add_points(all_spots[all_spots["gene"]==gene_to_plot.lower()][["r_um","c_um"]].values[::subsample,:], symbol='disc',
                                 name = gene_to_plot+"+",edge_color= [0,0,0,0], size = 1.5,
                                 face_color= colorlist[ cindex % len(colorlist)] , blending = "translucent", opacity = 0.8)
        else:
            spots = v.add_points(all_spots[all_spots["gene"]==gene_to_plot.lower()][["r_um","c_um"]].values[::subsample,:], symbol='disc',
                     name = gene_to_plot+"+",edge_color= [0,0,0,0], size = 2.,
                     face_color= colorlist[ cindex % len(colorlist)] , blending = "translucent", opacity = 0.7)
        cindex = cindex + 1

In [16]:
v.layers

[<Points layer 'Gad1+' at 0x10c9bd1d0>,
 <Points layer 'Gad2+' at 0x12ddec950>,
 <Points layer 'Slc17a7+' at 0x137967290>,
 <Points layer 'Cux2+' at 0x12e609090>,
 <Points layer 'Calb1+' at 0x12e44e150>,
 <Points layer 'Meis2+' at 0x12db0b710>,
 <Points layer 'Foxp2+' at 0x134432f90>,
 <Points layer 'Fezf2+' at 0x134432a90>,
 <Points layer 'Rorb+' at 0x13c7b22d0>,
 <Shapes layer 'Shapes' at 0x1377163d0>]

### annotate the layers in that viewer in a single `shapes` layer
#### Naming/order Convention:
0. outline of entire area
1. Layer 1
1. Layer 2/3
2. Layer 4
3. Layer 5
4. Layer 6
5. white matter
6. single line segment, pointing from pia to white matter 

#### annotate the layer locations in a list

In [9]:
shapes_layer = v.layers[-1].data[:-1]
column_points = v.layers[-1].data[-1]

In [15]:
if len(v.layers[-1].data) != 8:
    print("please annotate the 7 polygons and 1 line segment described above")
else:
    print("8 annotations")

8 annotations


In [73]:
# go get all the plotted points and note if they're in the annotated polygons
is_in_layer = {ii:[Polygon(p).intersects(Point(a))  for a  in all_spots[["x_um","y_um"]].values] for ii,p in enumerate(shapes_layer)}

In [76]:
all_spots["layer"] = "outside_VISp"
for k in is_in_layer.keys():
    all_spots.loc[is_in_layer[k],["layer"]] = annotation_names[k]


In [78]:
all_spots["layer"].unique()

array(['outside_VISp', 'VISp_wm', 'VISp_VI', 'VISp_V', 'VISp', 'VISp_IV',
       'VISp_II/III', 'VISp_I'], dtype=object)

In [20]:
# now the spots have depth coordinates, which will be useful for detected spots v. column depth
all_spots["depth_um"] = depth_coordinate(all_spots[["x_um","y_um"]].values, column_points)

In [79]:
all_spots

,Unnamed: 0,x_um,y_um,gene,depth_um,layer
0,0,1278.683956,6020.642260,Gad2,1445.701599,outside_VISp
1,1,1326.970330,6023.884788,Gad2,1403.168094,outside_VISp
2,2,1292.026844,6059.337093,Gad2,1413.731426,outside_VISp
3,3,1300.886241,6097.786264,Gad2,1385.680938,outside_VISp
4,4,1232.410068,6102.884182,Gad2,1440.812721,outside_VISp
...,...,...,...,...,...,...
3841407,3841407,3161.427603,5192.594981,Mup5,297.777986,outside_VISp
3841408,3841408,3099.698528,5221.596008,Mup5,334.424115,outside_VISp
3841409,3841409,3084.582240,5297.234605,Mup5,306.740686,outside_VISp
3841410,3841410,3054.192051,5342.142346,Mup5,308.399438,VISp_II/III


In [80]:
all_spots.to_csv("Allen_MERFISH_spots_with_anatomy.csv")

In [61]:
annotation_names = ["VISp", "VISp_I", "VISp_II/III", "VISp_IV", "VISp_V","VISp_VI", "VISp_wm", "column_axis"]


In [63]:
annotations = [mapping(Polygon(p))  for ii,p in enumerate(shapes_layer)]
annotations.append(mapping(LineString(column_points)))
for ii,ai in enumerate(annotations):
    ai["name"] = annotation_names[ii]

In [65]:
a = geojson.GeometryCollection(annotations)

In [66]:
with open("Allen_MERFISH_annotations_geo.json",'w') as w:
    geojson.dump(a,w)

In [7]:
with open("/Users/brian/work/data/stardist_rois.json",'r') as r:
    shapejson = geojson.load(r)

In [54]:
shapes_to_add = [0.1078*4.*np.array(geo["coordinates"][0][::5]) for geo in shapejson["geometries"]]
for f in shapes_to_add:
    f[:,0] = f[:,0]+1154
    f[:,1] = f[:,1]+4548  #6237

In [60]:
import imageio

In [62]:
egdapi = imageio.imread("/Users/brian/work/mFISH__002_DAPI_Overview-ds.tif")

In [59]:
print(10706.668188763051*0.1078)
print(44241.16940370426*0.1078 - 2048*.1078)

1154.178830748657
4548.423661719319


In [65]:
v.add_image(egdapi.T, scale = [4.0*0.1078, 4.0*0.1078], translate = [1154, 4548])

<Image layer 'Image [1]' at 0x15e81d350>

In [55]:
v.add_shapes(shapes_to_add ,
        shape_type='polygon',
        edge_width=1,
        edge_color='coral',
        face_color='royalblue',
        name='shapesjhjh')

<Shapes layer 'shapesjhjh' at 0x158d7d090>

In [18]:
shapes_to_add[33]

array([[1700., 1288.],
       [1699., 1290.],
       [1699., 1292.],
       [1697., 1294.],
       [1696., 1295.],
       [1694., 1296.],
       [1692., 1297.],
       [1690., 1298.],
       [1688., 1298.],
       [1686., 1298.],
       [1684., 1298.],
       [1682., 1297.],
       [1681., 1295.],
       [1679., 1294.],
       [1678., 1292.],
       [1677., 1290.],
       [1676., 1288.],
       [1676., 1285.],
       [1676., 1283.],
       [1677., 1280.],
       [1679., 1279.],
       [1682., 1278.],
       [1684., 1277.],
       [1686., 1277.],
       [1688., 1277.],
       [1690., 1278.],
       [1692., 1278.],
       [1694., 1279.],
       [1696., 1280.],
       [1698., 1282.],
       [1699., 1283.],
       [1699., 1286.],
       [1700., 1288.]])

In [38]:
1.03*63./60.

1.0815

In [37]:
40/(6660-5383)

0.031323414252153486

In [41]:
1888-1845

43

In [42]:
30

28

In [43]:
1817-1786

31

In [44]:
77-56

21